In [1]:

from importlib import import_module
import os
import sys
import argparse
import linecache
import uproot
import vector
import math
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
from tqdm import tqdm  # ✅ 진행률 표시
import glob

vector.register_awkward()

# 모든 .root 파일 경로 가져오기
path = "/data6/Users/achihwan/LRSM_tb_channel/SAMPLEPRODUCTION/WRtoNMutoMuMuTB-HadTop_MWR-5000_MN-00_13p6TeV/"
file_list = sorted(glob.glob(path + "*.root"))

cleaned_Nmother_muon_pt = []
cleaned_WR_mother_muon_pt = []
all_muon_pt = []

cleaned_ak4jet_pt = []
all_ak4jet_pt = []

cleaned_ak8jet_pt = []
all_ak8jet_pt = []

def overlap_removal(target, cleans, cut=0.4, dphi=False):
    mask = ak.ones_like(target["pt"], dtype=bool)
    for clean in cleans:
        pairs = ak.cartesian([target, clean], nested=True)
        # ΔR 계산
        raw = (pairs["0"].deltaphi(pairs["1"]) if dphi else pairs["0"].deltaR(pairs["1"]))
        delta = np.abs(raw)
        # 0인 값(=자기 자신)을 무시하기 위해 np.inf로 대체
        nonzero = ak.where(delta > 0, delta, np.inf)
        # 이제 nonzero 중 최소값을 취함 → 사실상 두 번째로 작은 원래 delta
        min_dr = ak.min(nonzero, axis=2)
        mask = mask & (min_dr > cut)
    return target[mask]

def find_closest_jet(obj, jets):
    # obj, jets: both are jagged arrays of shape (n_events,), each sublist = Momentum4D
    pairs = ak.cartesian([obj, jets], nested=True)          # shape: (n_events, N_obj, N_jet)
    dr = pairs["0"].deltaR(pairs["1"])                       # same shape
    # 이벤트별로 obj 하나당 가장 작은 ΔR 의 jet index
    closest_idx_per_obj = ak.argmin(dr, axis=2)              # shape = (n_events, N_obj)
    return jets[closest_idx_per_obj]





# tqdm으로 파일 리스트 순회
for sample in tqdm(file_list, desc="Processing ROOT files"):
    file = uproot.open(sample)
    events = file["Events"]

    keys = events.keys()

    vector.register_awkward()  

    lhe_pdgid = events["LHEPart_pdgId"].array()
    lhe_pt = events["LHEPart_pt"].array()
    lhe_eta = events["LHEPart_eta"].array()
    lhe_phi = events["LHEPart_phi"].array()
    lhe_mass = events["LHEPart_mass"].array()
    lhe_status = events["LHEPart_status"].array()

    ak4_eta = events["Jet_eta"].array()
    ak4_phi = events["Jet_phi"].array()
    ak4_pt = events["Jet_pt"].array()
    ak4_flavor = events["Jet_hadronFlavour"].array()
    ak4_mass = events["Jet_mass"].array()
    btag = events["Jet_btagPNetB"].array()

    ## Fatjet 
    fatjet_pt = events["FatJet_pt"].array()
    fatjet_eta = events["FatJet_eta"].array()
    fatjet_phi = events["FatJet_phi"].array()
    fatjet_mass = events["FatJet_mass"].array()

    gen_pdgid = events["GenPart_pdgId"].array()
    gen_pt = events["GenPart_pt"].array()
    gen_eta = events["GenPart_eta"].array()
    gen_phi = events["GenPart_phi"].array()
    gen_mass = events["GenPart_mass"].array()

    reco_muon_pt = events["Muon_pt"].array()
    reco_muon_eta = events["Muon_eta"].array()
    reco_muon_phi = events["Muon_phi"].array()
    reco_muon_mass = events["Muon_mass"].array()



    # mask 
    bottom_mask = (lhe_pdgid == 5) | (lhe_pdgid == -5)
    bottom_eta = lhe_eta[bottom_mask]
    bottom_phi = lhe_phi[bottom_mask]
    bottom_pt = lhe_pt[bottom_mask]
    bottom_mass = lhe_mass[bottom_mask]

    topmask = (gen_pdgid == 6) | (gen_pdgid == -6)
    top_eta = gen_eta[topmask]
    top_eta = top_eta[:, 0:1]  # top quark 1 = WR* mother
    top_phi = gen_phi[topmask]
    top_phi = top_phi[:, 0:1]
    top_pt = gen_pt[topmask]
    top_pt = top_pt[:, 0:1]  # top quark 1 = WR* mother
    top_mass = gen_mass[topmask]
    top_mass = top_mass[:, 0:1]  # top quark 1 = WR* mother

    bottompt = bottom_pt[:, 1:2]#bottom quark 1 = WR* mother
    bottometa = bottom_eta[:, 1:2]
    bottomphi = bottom_phi[:, 1:2]
    bottommass = bottom_mass[:, 1:2]

    muon_mask = (lhe_pdgid == 13) | (lhe_pdgid == -13)
    muon_pt = lhe_pt[muon_mask]
    muon_eta = lhe_eta[muon_mask]
    muon_phi = lhe_phi[muon_mask]
    muon_mass = lhe_mass[muon_mask]

    reco_muon_pt = reco_muon_pt[:]
    reco_muon_eta = reco_muon_eta[:]
    reco_muon_phi = reco_muon_phi[:]
    reco_muon_mass = reco_muon_mass[:] 


    n_mother_muon_mass = muon_mass[:,0:1] # muon 1 = WR* mother
    n_mother_muon_pt = muon_pt[:,0:1] # muon 1 = WR* mother
    n_mother_muon_eta = muon_eta[:,0:1]
    n_mother_muon_phi = muon_phi[:,0:1]

    WR_mother_muon_mass = muon_mass[:,1:2] # muon 2 = WR* mother
    WR_mother_muon_pt = muon_pt[:,1:2] # muon 2 = WR* mother
    WR_mother_muon_eta = muon_eta[:,1:2]
    WR_mother_muon_phi = muon_phi[:,1:2]  


    lhe_muons = ak.zip({
        "pt": muon_pt,
        "eta": muon_eta,
        "phi": muon_phi,
        "mass": muon_mass
    }, with_name = "Momentum4D")

    n_mother_muon = ak.zip({
        "pt": n_mother_muon_pt,
        "eta": n_mother_muon_eta,
        "phi": n_mother_muon_phi,
        "mass": n_mother_muon_mass
    }, with_name = "Momentum4D")

    WR_mother_muon = ak.zip({
        "pt": WR_mother_muon_pt,
        "eta": WR_mother_muon_eta,
        "phi": WR_mother_muon_phi,
        "mass": WR_mother_muon_mass
    }, with_name = "Momentum4D")

    reco_muons = ak.zip({
        "pt": reco_muon_pt,
        "eta": reco_muon_eta,
        "phi": reco_muon_phi,
        "mass": reco_muon_mass
    }, with_name = "Momentum4D")

    ak4 = ak.zip({
        "pt": ak4_pt,
        "eta": ak4_eta,
        "phi": ak4_phi,
        "mass": ak4_mass
    }, with_name = "Momentum4D")

    lhe_bottom = ak.zip({ ## bottom quark 1 = WR* mother
        "pt": bottompt,
        "eta": bottometa,
        "phi": bottomphi,
        "mass": bottommass
    }, with_name = "Momentum4D")

    gen_top = ak.zip({
        "eta": top_eta,
        "phi": top_phi,
        "pt": top_pt,
        "mass": top_mass
    }, with_name = "Momentum4D")

    fatjets = ak.zip({
        "pt": fatjet_pt,
        "eta": fatjet_eta,
        "phi": fatjet_phi,
        "mass": fatjet_mass
    }, with_name="Momentum4D")

    

    ak8jet = find_closest_jet(gen_top, fatjets)
    ak4jet = find_closest_jet(lhe_bottom, ak4)
    
    ## 1. 뮤온은 자기 자신끼리만 안 겹치면 그대로 사용
    overlap_nmother_cleaned = overlap_removal(n_mother_muon, [reco_muons]) #lepton lepton cleaning
    overlap_WR_mother_cleaned = overlap_removal(WR_mother_muon, [reco_muons]) #lepton lepton cleaning

    cleaned_Nmother_muon_pt.append(overlap_nmother_cleaned["pt"])
    cleaned_WR_mother_muon_pt.append(overlap_WR_mother_cleaned["pt"])
    all_muon_pt.append(reco_muons["pt"])
    # 2. b jet은 뮤온이랑만 안 겹치면 그대로 사용 
    overlap_ak4_cleaned = overlap_removal(ak4jet, [reco_muons, ak4])
    

    cleaned_ak4jet_pt.append(overlap_ak4_cleaned["pt"])
    all_ak4jet_pt.append(ak4jet["pt"])
    # 3. fatjet은 뮤온이랑 b jet이랑 안 겹쳐야 사용
    overlap_fatjet_cleaned = overlap_removal(ak8jet, [reco_muons , ak4 ,fatjets], cut=0.8)

    cleaned_ak8jet_pt.append(overlap_fatjet_cleaned["pt"])
    all_ak8jet_pt.append(fatjets["pt"])


    
    
    






Processing ROOT files: 0it [00:00, ?it/s]
